# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921691737979                   -0.69    5.8         
  2   -7.926165645383       -2.35       -1.22    1.0    213ms
  3   -7.926838204723       -3.17       -2.37    1.8    236ms
  4   -7.926861496385       -4.63       -3.05    2.6    249ms
  5   -7.926861647771       -6.82       -3.44    1.9    220ms
  6   -7.926861671900       -7.62       -3.84    1.9    245ms
  7   -7.926861680227       -8.08       -4.28    1.1    199ms
  8   -7.926861681692       -8.83       -4.74    2.0    223ms
  9   -7.926861681866       -9.76       -5.48    1.9    221ms
 10   -7.926861681871      -11.25       -5.82    2.1    233ms
 11   -7.926861681873      -11.93       -6.84    1.2    228ms
 12   -7.926861681873      -13.44       -7.53    3.0    262ms
 13   -7.926861681873   +  -15.05       -7.76    2.1    227ms
 14   -7.926861681873      -14.45       -8.79    1.4    206ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921673561086                   -0.69    5.9         
  2   -7.926164851933       -2.35       -1.22    1.0    213ms
  3   -7.926836406936       -3.17       -2.37    1.9    441ms
  4   -7.926861526867       -4.60       -3.02    2.9    292ms
  5   -7.926861636825       -6.96       -3.36    1.5    241ms
  6   -7.926861668175       -7.50       -3.75    1.9    250ms
  7   -7.926861680555       -7.91       -4.37    1.1    231ms
  8   -7.926861681791       -8.91       -4.89    2.2    619ms
  9   -7.926861681859      -10.17       -5.25    1.6    215ms
 10   -7.926861681871      -10.90       -5.88    1.8    218ms
 11   -7.926861681873      -11.91       -6.76    2.0    233ms
 12   -7.926861681873      -13.34       -7.26    2.8    254ms
 13   -7.926861681873   +  -14.75       -8.50    2.0    252ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921679368954                   -0.69    6.0         
  2   -7.926162419813       -2.35       -1.22    1.0    220ms
  3   -7.926841155100       -3.17       -2.37    1.9    264ms
  4   -7.926864907526       -4.62       -3.05    2.6    246ms
  5   -7.926865063700       -6.81       -3.45    1.9    229ms
  6   -7.926865083505       -7.70       -3.87    1.5    224ms
  7   -7.926865091351       -8.11       -4.28    1.6    198ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
